# Introduction

Implement YOLO usage as explained in the [reference article](https://towardsdatascience.com/yolo-object-detection-with-opencv-and-python-21e50ac599e9).

<br>

The model has been trained over the [COCO Dataset](https://cocodataset.org/#home).

In [2]:
# Import Standard Libraries
import cv2

In [ ]:
# Notebook variables
sample_image_path = './../images/dog_image_1.jpeg'
